In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import minimize
from datetime import datetime as dt

In [2]:
from eod import EodHistoricalData
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols

In [18]:
def heston_charfunc(phi,S0,K,v0,kappa,theta,sigma,rho,lambd,tau,r):
    a = kappa*theta
    b = kappa*lambd
    rspi = rho*sigma*phi*1j
    d = np.sqrt((rho*sigma*phi*1j-b)**2 + (phi*1j+phi**2)*sigma**2)
    g = (b-rspi+d)/(b-rspi-d)
    #characteristic function by components
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j)*((1-g*np.exp(d*tau))/(1-g))**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2+v0*(b-rspi+d)*((1-np.exp(d*tau))/(1-g*np.exp(d*tau)))/sigma**2)
    return exp1*term2*exp2

In [19]:
def integrand(phi,S0,K,v0,kappa,theta,sigma,rho,lambd,tau,r):
    args = (S0,K,v0,kappa,theta,sigma,rho,lambd,tau,r)
    numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args)-K*heston_charfunc(phi,*args)
    denominator = 1j*phi*K**(1j*phi)
    return numerator/denominator

Perform numerical integral

In [20]:
def heston_price_rec(S0,K,v0,kappa,theta,sigma,rho,lambd,tau,r):
    args = (S0,K,v0,kappa,theta,sigma,rho,lambd,tau,r)
    P,umax,N = 0,100,10000
    dphi = umax/N
    for i in range(1,N):
        phi = dphi*(2*i+1)/2 #mid-point to calculate height
        numerator = np.exp(r*tau)*hesto_charfunc(phi-1j,*args)-K*heston_charfunc(phi,*args)
        denominator = 1j*phi*K**(1j*phi)
        P+=dphi*numerator/denominator
    return np.real((S0-K*np.exp(-r*tau))/2+P/np.pi)

Use scipy integrate quad function

In [21]:
def heston_price(S0,K,v0,kappa,theta,sigma,rho,lambd,tau,r):
    args = (S0,K,v0,kappa,theta,sigma,rho,lambd,tau,r)
    real_integral,err = np.real(quad(integrand,0,100,args = args))
    return (S0-K*np.exp(-r*tau))/2+real_integral/np.pi

In [22]:
S0 = 100. #initial asset price
K = 100. # strike
v0 = 0.1 #initial variance
r = 0.03 # risk free rate
kappa = 1.5768 # rate of mean reversion of variance process
theta = 0.0398 # long-term mean variance
sigma = 0.3 # vol of vol
lambd = 0.575 # risk premium of variance
rho = -0.5711 # correlation between variance and stock process
tau = 1. #time to maturity

In [23]:
heston_price(S0,K,v0,kappa,theta,sigma,rho,lambd,tau,r)

/home/ohj/.local/lib/python3.10/site-packages/scipy/integrate/quadpack.py:463: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagse(func,a,b,args,full_output,epsabs,epsrel,limit)


13.716373159121495

Risk-free rate from US daily treasury par yield curve rates

In [26]:
yield_maturities = np.array([1/12,2/12,3/12,6/12,1,2,3,5,7,10,20,30])
yields = np.array([0.15,0.27,0.50,0.93,1.52,2.13,2.32,2.34,2.37,2.32,2.65,2.52]).astype(float)/100

In [27]:
curve_fit,status = calibrate_nss_ols(yield_maturities,yields)
curve_fit

NelsonSiegelSvenssonCurve(beta0=0.028391531922961333, beta1=-0.029279498864207682, beta2=0.025428219264836193, beta3=-0.01417407230854954, tau1=0.9922983277200589, tau2=4.78140939601007)

EOD historical data